In [1]:
print('hii prashant ')

hii prashant 


In [2]:
!pip install -q -U transformers
!pip install -q -U accelerate
!pip install -q -U datasets
!pip install -q -U trl

In [2]:
import torch, multiprocessing
from datasets import load_dataset
# from transformers
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments
)

from trl import SFTTrainer, SFTConfig, DPOTrainer, DPOConfig

/home/vivek/Documents/prashant/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [31]:
DEVICE = torch.cuda.get_device_name(0)

In [32]:
DEVICE

'NVIDIA GeForce RTX 3090'

In [33]:
major_version, minor_version = torch.cuda.get_device_capability()

In [34]:
major_version

8

In [35]:
minor_version

6

In [36]:
if major_version >=8:
    !pip install flash-attn
    attn_implementation = 'falsh_attentaion_2'
    print("your gpu is noob so its compataible")
else:
    print("Your device is noob pro max")


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 20.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.6.3-cp310-cp310-linux_x86_64.whl size=189291963 sha256=b0e1924ee6866db683f5cb72c1002f6d4ca39e9efd79efec1ffc75aaecb2a576
  Stored in directory: /tmp/pip-ephem-wheel-cache-y2_n7iaq/wheels/7e/e3/c3/89c7a2f3c4adc07cd1c675f8bb7b9ad4d18f64a72bccdfe826
Successfully built flash-attn
your gpu is noob so its compataible


In [2]:
model_name  = "HuggingFaceTB/SmolLM-135M"
from transformers import AutoConfig


config = AutoConfig.from_pretrained(model_name)

/home/vivek/Documents/prashant/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
config

LlamaConfig {
  "_name_or_path": "HuggingFaceTB/SmolLM-135M",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "eos_token_id": 0,
  "head_dim": 64,
  "hidden_act": "silu",
  "hidden_size": 576,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "max_position_embeddings": 2048,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 9,
  "num_hidden_layers": 30,
  "num_key_value_heads": 3,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.46.2",
  "use_cache": true,
  "vocab_size": 49152
}

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast = True)


In [7]:
tokenizer.special_tokens_map

{'bos_token': '<|endoftext|>',
 'eos_token': '<|endoftext|>',
 'unk_token': '<|endoftext|>',
 'additional_special_tokens': ['<|endoftext|>',
  '<|im_start|>',
  '<|im_end|>',
  '<repo_name>',
  '<reponame>',
  '<file_sep>',
  '<filename>',
  '<gh_stars>',
  '<issue_start>',
  '<issue_comment>',
  '<issue_closed>',
  '<jupyter_start>',
  '<jupyter_text>',
  '<jupyter_code>',
  '<jupyter_output>',
  '<jupyter_script>',
  '<empty_output>']}

In [14]:
tokenizer.pad_token = tokenizer.additional_special_tokens[2]
tokenizer.pad_token_id = 2

tokenizer.padding_side = "left"

In [15]:
dataset_train_sft = load_dataset('HuggingFaceH4/ultrachat_200k', split='train_sft')
dataset_test_sft = load_dataset('HuggingFaceH4/ultrachat_200k', split='test_sft')



Generating test_gen split: 100%|██████████| 28304/28304 [00:00<00:00, 50364.18 examples/s]


In [16]:
dataset_train_sft[0]

{'prompt': "These instructions apply to section-based themes (Responsive 6.0+, Retina 4.0+, Parallax 3.0+ Turbo 2.0+, Mobilia 5.0+). What theme version am I using?\nOn your Collections pages & Featured Collections sections, you can easily show the secondary image of a product on hover by enabling one of the theme's built-in settings!\nYour Collection pages & Featured Collections sections will now display the secondary product image just by hovering over that product image thumbnail.\nDoes this feature apply to all sections of the theme or just specific ones as listed in the text material?",
 'prompt_id': 'f0e37e9f7800261167ce91143f98f511f768847236f133f2d0aed60b444ebe57',
 'messages': [{'content': "These instructions apply to section-based themes (Responsive 6.0+, Retina 4.0+, Parallax 3.0+ Turbo 2.0+, Mobilia 5.0+). What theme version am I using?\nOn your Collections pages & Featured Collections sections, you can easily show the secondary image of a product on hover by enabling one of 

In [17]:
dataset_test_sft[0]

{'prompt': 'How does the author propose to fix the problem of science alienation in our educational system? What changes does she suggest should be made to science education? Answer according to: Science education should be split into two tracks.\nSplit K-12 science education into two tracks, for majors and nonmajors.\nThose who want to specialize in science could take math and complex chemistry. Nonmajors would focus on science of the everyday—things like kitchen chemistry and CSI-style crime investigations.\nSome years ago, when I was working as a newspaper science writer in California, I fell into a rather idle conversation with a physicist on the subject of science education. Idle for him, at least, because what he said—the way he defined the American system of teaching K-12 students about science—has stayed with me since.\nThis conversation has returned to me many times over the years. On the day my older son, who spent his childhood joyfully chasing insects and reading natural hi

In [ ]:
# major_version, minor_version = torch.cuda.get_device_capability()
# if major_version >= 8:
#   !pip install flash-attn
#   attn_implementation='flash_attention_2'
#   print("Your GPU is compatible with FlashAttention.")
# else:
#   attn_implementation='eager'
#   print("Your GPU is not compatible with FlashAttention.")

chat = [
  {"role": "user", "content": "Hello, how are you?"},
  {"role": "assistant", "content": "I'm doing great. How can I help you today?"},
  {"role": "user", "content": "I'd like to show off how chat templating works!"},
]

tokenizer.apply_chat_template(chat, tokenize=False)


model_name = "HuggingFaceTB/SmolLM-135M"
#Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = "<|im_end|>"
tokenizer.pad_token_id = 2
tokenizer.padding_side = 'left' #Necessary for FlashAttention compatibility

In [19]:
model = AutoModelForCausalLM.from_pretrained(
          model_name, attn_implementation="eager", device_map={"": 0}
)
model.gradient_checkpointing_enable(gradient_checkpointing_kwargs={'use_reentrant':True})

In [31]:
dataset_train_sft = load_dataset("HuggingFaceH4/ultrachat_200k", split="train_sft")
dataset_test_sft = load_dataset("HuggingFaceH4/ultrachat_200k", split="test_sft[:5%]")


In [32]:
dataset_train_sft[0]

{'prompt': "These instructions apply to section-based themes (Responsive 6.0+, Retina 4.0+, Parallax 3.0+ Turbo 2.0+, Mobilia 5.0+). What theme version am I using?\nOn your Collections pages & Featured Collections sections, you can easily show the secondary image of a product on hover by enabling one of the theme's built-in settings!\nYour Collection pages & Featured Collections sections will now display the secondary product image just by hovering over that product image thumbnail.\nDoes this feature apply to all sections of the theme or just specific ones as listed in the text material?",
 'prompt_id': 'f0e37e9f7800261167ce91143f98f511f768847236f133f2d0aed60b444ebe57',
 'messages': [{'content': "These instructions apply to section-based themes (Responsive 6.0+, Retina 4.0+, Parallax 3.0+ Turbo 2.0+, Mobilia 5.0+). What theme version am I using?\nOn your Collections pages & Featured Collections sections, you can easily show the secondary image of a product on hover by enabling one of 

In [28]:
def format_dataset(example):
    """Format the dataset into the expected conversation format"""
    # The messages are already in the correct format, we just need to normalize the roles
    messages = [
        {
            "role": msg["role"].replace("human", "user"),  # Convert 'human' to 'user' if needed
            "content": msg["content"]
        }
        for msg in example["messages"]
    ]
    return {"messages": messages}



# Load datasets
dataset_train_sft = load_dataset("HuggingFaceH4/ultrachat_200k", split="train_sft")
dataset_test_sft = load_dataset("HuggingFaceH4/ultrachat_200k", split="test_sft[:5%]")

dataset_train_sft = dataset_train_sft.map(
    format_dataset,
    remove_columns=dataset_train_sft.column_names
)
dataset_test_sft = dataset_test_sft.map(
    format_dataset,
    remove_columns=dataset_test_sft.column_names
)


Map: 100%|██████████| 1156/1156 [00:00<00:00, 4910.65 examples/s]


In [10]:
model = AutoModelForCausalLM.from_pretrained(
          model_name, attn_implementation="eager", device_map={"": 0}
)
model.gradient_checkpointing_enable(gradient_checkpointing_kwargs={'use_reentrant':True})

In [11]:
training_arguments = SFTConfig(
        output_dir="./sft_smollm_135M/",
        eval_strategy="steps",
        do_eval=True,
        optim="adamw_torch",
        per_device_train_batch_size=8,
        gradient_accumulation_steps=2,
        per_device_eval_batch_size=8,
        log_level="debug",
        save_steps=500,
        logging_steps=50,
        learning_rate=2e-5,
        fp16= not torch.cuda.is_bf16_supported(),
        bf16= torch.cuda.is_bf16_supported(),
        eval_steps=50,
        max_steps=4000,
        warmup_steps=30,
        max_seq_length=2048,
        lr_scheduler_type="linear",
)

In [13]:
trainer = SFTTrainer(
        model=model,
        train_dataset=dataset_train_sft,
        eval_dataset=dataset_test_sft,
        tokenizer=tokenizer,
        args=training_arguments,
)

trainer.train()

Generating train split: 0 examples [00:00, ? examples/s]


ValueError: Error occurred while packing the dataset. Make sure that your dataset has enough samples to at least yield one packed sequence.

In [27]:
print("\nFirst example:", dataset_train_sft[0])



First example: {'prompt': "These instructions apply to section-based themes (Responsive 6.0+, Retina 4.0+, Parallax 3.0+ Turbo 2.0+, Mobilia 5.0+). What theme version am I using?\nOn your Collections pages & Featured Collections sections, you can easily show the secondary image of a product on hover by enabling one of the theme's built-in settings!\nYour Collection pages & Featured Collections sections will now display the secondary product image just by hovering over that product image thumbnail.\nDoes this feature apply to all sections of the theme or just specific ones as listed in the text material?", 'prompt_id': 'f0e37e9f7800261167ce91143f98f511f768847236f133f2d0aed60b444ebe57', 'messages': [{'content': "These instructions apply to section-based themes (Responsive 6.0+, Retina 4.0+, Parallax 3.0+ Turbo 2.0+, Mobilia 5.0+). What theme version am I using?\nOn your Collections pages & Featured Collections sections, you can easily show the secondary image of a product on hover by en

In [3]:
model_name = "HuggingFaceTB/SmolLM-135M"
#Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = "<|im_end|>"
tokenizer.pad_token_id = 2
tokenizer.padding_side = 'left' #Necessary for FlashAttention compatibility

In [4]:
chat_template = """
Human: {human_input}
Assistant: {assistant_output}
"""
tokenizer.chat_template = chat_template

In [5]:
dataset_train_sft = load_dataset("HuggingFaceH4/ultrachat_200k", split="train_sft")
dataset_test_sft = load_dataset("HuggingFaceH4/ultrachat_200k", split="test_sft[:5%]")


In [6]:
model_name = "HuggingFaceTB/SmolLM-135M"
#Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = "<|im_end|>"
tokenizer.pad_token_id = 2
tokenizer.padding_side = 'left' #Necessary for FlashAttention compatibility"

In [7]:
training_arguments = SFTConfig(
        output_dir="./sft_smollm_135M/",
        eval_strategy="steps",
        do_eval=True,
        optim="adamw_torch",
        per_device_train_batch_size=8,
        gradient_accumulation_steps=2,
        per_device_eval_batch_size=8,
        log_level="debug",
        save_steps=500,
        logging_steps=50,
        learning_rate=2e-5,
        fp16= not torch.cuda.is_bf16_supported(),
        bf16= torch.cuda.is_bf16_supported(),
        eval_steps=50,
        max_steps=4000,
        warmup_steps=30,
        max_seq_length=2048,
        lr_scheduler_type="linear",
)

In [8]:
trainer = SFTTrainer(
    model = model,
    train_dataset = dataset_train_sft,
    eval_dataset = dataset_test_sft,
    tokenizer = tokenizer,
    args = training_arguments,
)
trainer.train()


NameError: name 'model' is not defined

In [ ]:
aaaaukllp][bbcvdec dnn rtyyuuip[[8+++-\    n ]]]